# 

In [19]:
import torch
import numpy as np
import os

import time

import sys
sys.path.append("src/")

from qcd_ml.nn.lptc import v_LPTC
from qcd_ml.qcd.dirac import dirac_wilson_clover
#from qcd_ml.compat.gpt import lattice2ndarray, ndarray2lattice
from qcd_ml.util.solver import GMRES_torch
from qcd_ml.util.qcd.multigrid import ZPP_Multigrid
from qcd_ml.base.paths import v_evaluate_path, v_ng_evaluate_path, v_reverse_evaluate_path, PathBuffer
from qcd_ml.base.operations import v_spin_transform, v_ng_spin_transform, v_gauge_transform


vec = torch.complex(
        torch.randn(8, 8, 8, 16, 4, 3, dtype=torch.double)
        , torch.randn(8, 8, 8, 16, 4, 3, dtype=torch.double))

U = torch.tensor(np.load(os.path.join("test", "assets","1500.config.npy")))

U_smeared = torch.load("U_smeared.pt")
U_smeared = [torch.tensor(np.array(Us)) for Us in U_smeared]

import itertools
def get_paths_lexicographic(block_size):
    paths = []
    for position in itertools.product(*(range(bs) for bs in block_size)):
        path = sum([[(mu, -1)] for mu, n in enumerate(position) for _ in range(n)], start=[])
        paths.append(path)
    return paths

def get_paths_reverse_lexicographic(block_size):
    return [list(reversed(pth)) for pth in get_paths_lexicographic(block_size)]

def get_paths_one_step_lexicographic(block_size):
    paths = []
    for position in itertools.product(*(range(bs) for bs in block_size)):
        path = []
        pos = np.array(position)
        while pos.any():
            for mu in range(pos.shape[0]):
                if pos[mu] > 0:
                    path.append((mu, -1))
                    pos[mu] -= 1
        paths.append(path)
    return paths
def get_paths_one_step_reverse_lexicographic(block_size):
    return [list(reversed(pth)) for pth in get_paths_one_step_lexicographic(block_size)]


def path_get_orig_point(path):
    point = [0] * 4
    for mu, nhops in path:
        point[mu] -= nhops
    return point



/tmp/ipykernel_160777/2789506356.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  U_smeared = torch.load("U_smeared.pt")


In [14]:
class v_ProjectLayer(torch.nn.Module):
    def __init__(self, gauges_and_paths, L_fine, L_coarse):
        super().__init__()
        self.path_buffers = [[PathBuffer(Ui, pij) for pij in pi] for Ui, pi in gauges_and_paths]
        self.weights = torch.nn.Parameter(
                torch.randn(len(gauges_and_paths), *tuple(U.shape[1:-2]), 4, 4, dtype=torch.cdouble)
                )
        self.L_fine = L_fine
        self.L_coarse = L_coarse
        self.block_size = [lf // lc for lf, lc in zip(L_fine, L_coarse)]

        self.base_points = [[path_get_orig_point(pb.path) for pb in path_buffers] for path_buffers in self.path_buffers]
        # This keeps a gauge field for every point on the lattice
        # such that we can use this exact field to transform the
        # base_points before summing them up.
        self.gauge_fields = torch.zeros(len(gauges_and_paths), U.shape[1:], dtype=torch.cdouble)

        identity = torch.tensor([[1, 0, 0], [0, 1, 0], [0, 0, 1]], dtype=torch.cdouble)
        for i, gpi in enumerate(self.path_buffers):
            for j, pb in enumerate(gpi):
                base_point = self.base_points[i,j]
                if pb._is_identity:
                    self.gauge_fields[i
                            , base_point[0]::self.block_size[0]
                            , base_point[1]::self.block_size[1]
                            , base_point[2]::self.block_size[2]
                            , base_point[3]::self.block_size[3]] = identity
                else:
                    self.gauge_fields[i
                            , base_point[0]::self.block_size[0]
                            , base_point[1]::self.block_size[1]
                            , base_point[2]::self.block_size[2]
                            , base_point[3]::self.block_size[3]] = pb.accumulated_U[base_point[0]::self.block_size[0]
                                                                        , base_point[1]::self.block_size[1]
                                                                        , base_point[2]::self.block_size[2]
                                                                        , base_point[3]::self.block_size[3]]
        def v_project(self, features_in):
            before_pool = torch.zeros(features_in.shape[0], self.gauge_fields.shape[0], *features_in.shape[1:]
                                     , dtype=torch.cdouble)
            for i, fea_i in enumerate(features_in):
                for j, (gfj, wj) in enumerate(zip(self.gauge_fields, self.weights)):
                    before_pool[i,j] = v_spin_transform(wj, v_gauge_transform(gfj, fea_i))

            # FIXME: use scatter_add_ to pool inside the block.
            return torch.sum(before_pool, axis=1)





SyntaxError: incomplete input (1946293122.py, line 16)

In [15]:
pb = PathBuffer(U, [(0,-1), (1,-3), (2,-2)])
orig_p = path_get_orig_point(pb.path)

In [16]:
orig_p

[1, 3, 2, 0]

In [17]:
psi = torch.zeros_like(vec)
psi[*tuple(orig_p), 0, 0] = 1

In [18]:
pb.v_transport(psi)[0,0,0,0]

tensor([[ 0.5171+0.3549j,  0.2084+0.2717j, -0.6081-0.3459j],
        [ 0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j],
        [ 0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j],
        [ 0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j]],
       dtype=torch.complex128)